In [4]:
from preparation import process_caption_index_helper, invoke_emotion_to_dataset, invoke_edited_to_dataset, load_caption

In [5]:
import json
mongo_dump_path = './dataset/dump/041619.json'
path = './dataset/caption_bc.json'
flickr_folder = './dataset/flickr10k'
coco_folder = './dataset/mscoco'
save_path = './dataset/cache/caption_index.helper'
image_id_to_idx = process_caption_index_helper(path, flickr_folder, coco_folder, save_path)
for mode in ['happy', 'sad', 'angry']:
    invoke_emotion_to_dataset(mongo_dump_path,image_id_to_idx, flickr_folder, 'flickr', mode)
invoke_edited_to_dataset(mongo_dump_path, flickr_folder)

In [6]:
with open(path, 'r') as f:
    lines = f.readlines()
    for l in lines:
        print(l)
        break

{"_id":{"$oid":"5c1fa377260e573d840dbbca"},"captions":[{"id":"Seorang lelaki mengenakan celana pendek dan kemeja Hawaii bersandar di atas rel perahu pilot, dengan kabut dan pegunungan di latar belakang.","en":"A man in shorts and a Hawaiian shirt leans over the rail of a pilot boat , with fog and mountains in the background .","caption_id":464109},{"id":"Seorang pria muda yang tergantung di sisi perahu, yang dalam kondisi seperti kabut berguling di atas bukit di belakangnya.","en":"A young man hanging over the side of a boat , which is in a like with fog rolling over a hill behind it .","caption_id":464110},{"id":"Seorang pria bersandar di sisi perahu biru dan putih saat ia duduk di badan air.","en":"A man is leaning off of the side of a blue and white boat as it sits in a body of water .","caption_id":464111},{"id":"Seorang pria menaiki perahu kecil di pelabuhan, dengan kabut dan pegunungan di latar belakang.","en":"A man riding a small boat in a harbor , with fog and mountains in the

In [3]:
train, val = load_caption(flickr_folder, lang='edited')

FileNotFoundError: [Errno 2] No such file or directory: './dataset/flickr10k/test.txt'

In [ ]:
filenames_train, captions_train = train

In [ ]:
captions_train['sad'][99:104]

In [ ]:
len(filenames_train['sad'])

In [ ]:
target = '5588403950.jpg'
for filename, caption in zip(filenames_train['happy'], captions_train['happy']):
    if filename == target:
        print(caption)
        break

In [ ]:
for filename, caption in zip(filenames_train['angry'], captions_train['angry']):
    if filename == target:
        print(caption)
        break

In [ ]:
from preparation import convert_mongo, split_and_save, load_caption

path = './dataset'
flickr_folder = path + '/flickr10k'
coco_folder = path + '/mscoco'

# # convert mongo to caption dict
# caption_flickr, caption_coco = convert_mongo(path=path + '/caption_bc.json')

# # save
# with open(flickr_folder + '/caption.json', 'w') as f:
#     json.dump(caption_flickr, f)
# with open(coco_folder + '/caption.json', 'w') as f:
#     json.dump(caption_coco, f)

# split_and_save(caption_flickr, flickr_folder)
# split_and_save(caption_coco, coco_folder)

train, val = load_caption(flickr_folder)
filenames_train, captions_train = train
filenames_val, captions_val = val
num_images_train = len(filenames_train['factual'])
num_images_train

In [ ]:
for caption in zip(filenames_train, captions_train):
    print(caption)

In [11]:
from tokenizer import mark_captions, flatten, TokenizerWrap, mark_start, mark_end

Using TensorFlow backend.


In [59]:
%%time
num_words = 10000
modes = ['happy', 'sad', 'angry']
captions_train_flat_all = []
for mode in ['factual']:
    captions_train_marked = mark_captions(captions_train[mode])
    captions_train_flat = flatten(captions_train_marked)
    tokenizer = TokenizerWrap(texts=captions_train_flat, num_words=num_words)
    # remove oov words
    tmp = tokenizer.texts_to_sequences(captions_train_flat)
    captions_train_flat = tokenizer.sequences_to_texts(tmp)
    captions_train_flat_all.extend(captions_train_flat)
tokenizer = TokenizerWrap(texts=captions_train_flat_all)

CPU times: user 2.52 s, sys: 20.5 ms, total: 2.54 s
Wall time: 2.56 s


In [60]:
%%time
captions_train_marked = mark_captions(captions_train['factual'])
captions_train_flat = flatten(captions_train_marked)
tokens_train = tokenizer.captions_to_tokens(captions_train_marked)

CPU times: user 800 ms, sys: 10.6 ms, total: 810 ms
Wall time: 813 ms


In [61]:
last = len(tokenizer.word_index)

In [62]:
tokenizer.index_to_word[last]

'mencoleknya'

In [63]:
from nltk.translate.bleu_score import sentence_bleu

In [64]:
references = ['it is a gloomy day', 'it is a sunny day village']
hypotesis = 'it is a great day'
references = [reference.split(' ') for reference in references]
hypotesis = hypotesis.split(' ')

In [65]:
sentence_bleu(references, hypotesis, weights=(0.33, 0.33, 0.33, 0))*100

/Users/dery/Documents/final_project/13515097-stylenet/env/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


51.43157015215767

In [75]:
all_filenames = filenames_train['factual']
all_captions = captions_train_marked
len(all_filenames)

9000

In [76]:
# Using heuristic to sort captions from the largest word frequency

pair_all_captions = []
for i in range(len(all_filenames)):
    caps = all_captions[i]
    filenames = all_filenames[i]
    total = 0
    length = 0
    for cap in tokenizer.texts_to_sequences(caps):
        for word_i in cap:
            length += 1
            word = tokenizer.index_to_word[word_i]
            total += tokenizer.word_counts[word]
    pair_all_captions.append((total/length, i))
pair_all_captions = sorted(pair_all_captions, key=lambda x: x[0], reverse=True)

In [77]:
val_num = len(all_filenames) * 1 // 10
test_num = len(all_filenames) * 1 // 10
train_indexes = []
val_indexes = []
test_indexes = []

In [78]:
# Get all proper indexes for train, val, and test
# Train must have all words within dataset

word_counts = tokenizer.word_counts

for total, index in pair_all_captions:
    caps = all_captions[index]
    is_safe = True
    for cap in tokenizer.texts_to_sequences(caps):
        if not is_safe:
            break
        for word_i in cap:
            if not is_safe:
                break
            word = tokenizer.index_to_word[word_i]
            cnt = word_counts[word]
            word_counts[word] -= 1
            is_safe = cnt>1
    if is_safe and len(val_indexes)<val_num:
        val_indexes.append(index)
    elif is_safe and len(test_indexes)<test_num:
        test_indexes.append(index)
    else:
        train_indexes.append(index)

In [82]:
all_words = []
for i in train_indexes:
    caps = all_captions[i]
    for cap in tokenizer.texts_to_sequences(caps):
        all_words.extend(cap)
for i in val_indexes:
    caps = all_captions[i]
    for cap in tokenizer.texts_to_sequences(caps):
        all_words.extend(cap)
for i in test_indexes:
    caps = all_captions[i]
    for cap in tokenizer.texts_to_sequences(caps):
        all_words.extend(cap)
all_words = list(set(all_words))

In [83]:
len(all_words)

9630

In [84]:
len(tokenizer.word_index)

9630

In [1]:
from preparation import convert_mongo
import json

mongo_dump_path = './dataset/dump/041619.json'
path = './dataset/caption_bc.json'
flickr_folder = './dataset/flickr10k'
coco_folder = './dataset/mscoco'
save_path = './dataset/cache/caption_index.helper'
caption_flickr, caption_coco = convert_mongo(path)

with open(flickr_folder + '/captions.json', 'r') as f:
    caption_flickr = json.load(f)

Using TensorFlow backend.


In [2]:
from preparation import invoke_emotion_to_dataset, invoke_edited_to_dataset, load_caption

for mode in ['happy', 'sad', 'angry']:
    invoke_emotion_to_dataset(mongo_dump_path, flickr_folder, 'flickr', mode)

In [3]:
all_filenames = {
    'factual': [],
    'happy': [],
    'sad': [],
    'angry': []
}
all_captions = {
    'factual': [],
    'happy': [],
    'sad': [],
    'angry': []
}
for mode in ['happy', 'sad', 'angry']:
    for data in caption_flickr:
        if data['emotions'].get(mode):
            all_filenames[mode].append(data['filename'])
            all_captions[mode].append([data['emotions'][mode]])
all_filenames['factual'] = [data['filename'] for data in caption_flickr]
all_captions['factual'] = [[caption['id'] for caption in data['captions']] for data in caption_flickr]

In [4]:
from tokenizer import mark_captions, flatten, TokenizerWrap, mark_start, mark_end

In [5]:
%%time
modes = ['happy', 'sad', 'angry']
captions_flat_all = []
for mode in ['factual'] + modes:
    captions_marked = mark_captions(all_captions[mode])
    captions_flat = flatten(captions_marked)
    tokenizer = TokenizerWrap(texts=captions_flat)
    # remove oov words
    tmp = tokenizer.texts_to_sequences(captions_flat)
    captions_flat = tokenizer.sequences_to_texts(tmp)
    captions_flat_all.extend(captions_flat)
tokenizer = TokenizerWrap(texts=captions_flat_all)

CPU times: user 2.99 s, sys: 29.5 ms, total: 3.02 s
Wall time: 3.04 s


In [6]:
from preparation import split_dataset, save_dataset

modes = ['happy', 'sad', 'angry']
for mode in ['factual'] + modes:
    train_indexes, val_indexes, test_indexes = split_dataset(all_filenames[mode], all_captions[mode], flickr_folder + '/' + mode, tokenizer, 10, 10)
    save_dataset(all_filenames[mode], all_captions[mode], flickr_folder + '/' + mode, train_indexes, val_indexes, test_indexes)

In [13]:
from preparation import load_caption

train, val, test = load_caption(flickr_folder + '/angry')

In [14]:
len(train[1])

477